Ошибки посмотреть под ячейками  
На веб морде эйрфлоу предупреждение: "The scheduler does not appear to be running. The DAGs list may not update, and new tasks will not be scheduled".

Код из статей:
- [Как запустить Apache AirFlow в Google Colab с ngrok](https://bigdataschool.ru/blog/how-to-launch-airflow-on-colab-with-ngrok.html)
- [Как запустить DAG AirFlow в Google Colab: простой пример](https://bigdataschool.ru/blog/how-to-run-user-dag-airflow-in-colab-with-ngrok.html) (пока код сюда не внес)

In [5]:
#############################ячейка №1 в Google Colab###########################
#Установка Apache Airflow и инициализация базы данных
!pip install apache-airflow
!airflow db init
## initdb - было так в оригинале, но пишет об ошибке и замене команды
#Установка инструмента ngrok для создания безопасного туннеля
#для доступа к веб-интерфейсу Airflow из любого места
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!pip install pyngrok
#импорт модулей
from pyngrok import ngrok
import sys
import os
import datetime
from airflow import DAG
from airflow.operators.bash import BashOperator

DB: sqlite:////root/airflow/airflow.db
[2023-10-21T06:28:46.127+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-10-21T06:28:46.128+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-10-21T06:28:46.626+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-10-21T06:28:46.627+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-10-21T06:28:46.629+0000] {db.py:1620} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [unusual_prefix_4c1c0063a36d033a56d4343efc723c3a45c1bd59_example_local_kubernetes_executor] Could not import DAGs in example_local_kubernetes_executor.py
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/airflow/example_dags/example_local_kubernetes_executor.py", line 37, in <module>
    from kubernetes.client import models as k8s
ModuleNotFoundError: No module named 'kubernet

In [6]:
#######################ячейка №2 в Google Colab########################
#Задание переменной auth_token для аутентификации в сервисе ngrok.
auth_token = "2WhKx82LS1maMJ2dMagXgh9ipsb_3fcJGdvLkWMHihaHRpLju" #@param {type:"string"}
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel
# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar
# and copy the API key
#Аутентификация в сервисе ngrok с помощью auth_token
os.system(f"ngrok authtoken {auth_token}")
#Запуск ngrok, который создаст публичный URL для сервера через туннель
#для доступа к веб-интерфейсу Airflow из любого места.
#addr="8888" указывает на порт, на котором запущен веб-сервер Airflow,
#а proto="http" указывает на использование протокола HTTP
public_url = ngrok.connect(addr="8888", proto="http")
#Вывод публичного URL для доступа к веб-интерфейсу Airflow
print("Адрес Airflow GUI:", public_url)

[2023-10-21T06:34:38.980+0000] {ngrok.py:276} INFO - Opening tunnel named: http-8888-4e9147c0-cf09-45e4-9cbf-0d43881996ae
[2023-10-21T06:34:39.011+0000] {process.py:99} INFO - t=2023-10-21T06:34:39+0000 lvl=info msg="no configuration paths supplied"
[2023-10-21T06:34:39.015+0000] {process.py:99} WARNING - t=2023-10-21T06:34:39+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/root/.config/ngrok/ngrok.yml legacy_path=/root/.ngrok2/ngrok.yml
[2023-10-21T06:34:39.019+0000] {process.py:99} INFO - t=2023-10-21T06:34:39+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
[2023-10-21T06:34:39.023+0000] {process.py:99} INFO - t=2023-10-21T06:34:39+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
[2023-10-21T06:34:39.026+0000] {process.py:99} INFO - t=2023-10-21T06:34:39+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
[2023-10-21T06:34:39.147+0000] {p

In [ ]:
## выше адрес вида https://2890-34-69-125-71.ngrok-free.app,
## по которому  зайдемв веб-интерфейс Apache AirFlow

In [8]:
#############################ячейка №3 в Google Colab#########################
!airflow db init #Инициализация базы данных Airflow  ## это же было в первой ячейке
!airflow db upgrade #Обновление базы данных Airflow
## !airflow upgradedb - в оригинале было, но ворнинги требуют новый синтаксис
#Создание нового пользователя в Apache Airflow с именем пользователя anna, именем Anna, фамилией Anna, адресом электронной почты anna.doe@example.com и паролем password.
#Этот пользователь будет иметь роль Admin, которая дает полный доступ к интерфейсу Airflow.
!airflow users create --username anna --firstname Anna --lastname Anna --email anna.doe@example.com --role Admin --password password

DB: sqlite:////root/airflow/airflow.db
[2023-10-21T06:37:49.924+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-10-21T06:37:49.925+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-10-21T06:37:50.418+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-10-21T06:37:50.418+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-10-21T06:37:50.420+0000] {db.py:1620} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [unusual_prefix_4c1c0063a36d033a56d4343efc723c3a45c1bd59_example_local_kubernetes_executor] Could not import DAGs in example_local_kubernetes_executor.py
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/airflow/example_dags/example_local_kubernetes_executor.py", line 37, in <module>
    from kubernetes.client import models as k8s
ModuleNotFoundError: No module named 'kubernet

In [9]:
##########################ячейка №5 в Google Colab#######################
#запуск веб-сервера Apache Airflow на порту 8888. Веб-сервер Airflow предоставляет пользовательский интерфейс
#для управления DAG’ами,
#просмотра логов выполнения задач, мониторинга прогресса выполнения
!airflow webserver --port 8888

  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
Running the Gunicorn Server with:
Workers: 4 sync
Host: 0.0.0.0:8888
Timeout: 120
Logfiles: - -
Access Logformat: 
/usr/local/lib/python3.10/dist-packages/flask_limiter/extension.py:336 UserWarning: Using the in-memory storage for tracking rate limits as no storage was explicitly specified. This is not recommended for production use. See: https://flask-limiter.readthedocs.io#configuring-a-storage-backend for documentation about configuring the storage backend.
[2023-10-21 06:38:35 +0000] [7328] [INFO] Starting gunicorn 21.2.0
[2023-10-21 06:38:35 +0000] [7328] [INFO] Listening at: http://0.0.0.0:8888 (7328)
[2023-10-21 06:38:35 +0000] [7328] [INFO] Using worker: sync
[2023-10-21 06:38:35 +0000] [7346] [INFO] Booting worker with pid: 7346
[2023-10-21 06:3

In [ ]:
## Теперь можно войти в веб-интерфейс Apache AirFlow,
## по ссылке, ранее сгенерированной утилитой ngrok.
## Надо ввести логин и пароль пользователя, созданного ранее: anna, password

In [10]:
############################ячейка №6 в Google Colab############################
!pkill -f ngrok #поиск и завершение процесса, который запущен с помощью команды ngrok
# отключение публичного URL, созданного с помощью ngrok для веб-сервера Airflow
ngrok.disconnect(public_url=public_url)